<a href="https://colab.research.google.com/github/humanet1603/krx_using_api/blob/main/NVDA_ma5_ma10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

# 파일 로드
file_path = '/content/엔비디아3분데이터.csv'
data = pd.read_csv(file_path,encoding='cp949')

# 상위 5개 데이터 출력
data.head()


In [ ]:
# 이동 평균선(MA) 계산
data['MA5'] = data['종가'].rolling(window=5).mean()
data['MA10'] = data['종가'].rolling(window=10).mean()

# MA5와 MA10 계산 결과 확인
data[['일자 / 시간', '종가', 'MA5', 'MA10']].head(15)


In [ ]:
# 매수 및 매도 신호 생성
data['Position'] = 0  # 매수(1), 매도(-1), 보유(0) 상태
data.loc[data['MA5'] > data['MA10'], 'Position'] = 1  # MA5가 MA10보다 클 때 매수
data.loc[data['MA5'] < data['MA10'], 'Position'] = -1  # MA5가 MA10보다 작을 때 매도

# 매수 및 매도 위치 변경 포인트 찾기
data['Signal'] = data['Position'].diff()

# 백테스팅을 위한 수익률 계산
data['Returns'] = data['종가'].pct_change()
data['Strategy'] = data['Returns'] * data['Position'].shift(1)  # 포지션 변경 후 수익 계산

# 전체 전략 수익률 계산
data['Cumulative Market Returns'] = (1 + data['Returns']).cumprod()
data['Cumulative Strategy Returns'] = (1 + data['Strategy']).cumprod()

# 최종 결과 확인
backtesting_results = data[['일자 / 시간', '종가', 'MA5', 'MA10', 'Position', 'Signal', 'Returns', 'Strategy', 'Cumulative Market Returns', 'Cumulative Strategy Returns']]
backtesting_results.tail()


In [ ]:
import pandas as pd

# 데이터 로드
data = pd.read_csv('/content/엔비디아3분데이터.csv', encoding='cp949')
data['종가'] = data['종가'].astype(float)
data['Returns'] = data['종가'].pct_change()

# MACD 계산
data['EMA12'] = data['종가'].ewm(span=12, adjust=False).mean()
data['EMA26'] = data['종가'].ewm(span=26, adjust=False).mean()
data['MACD'] = data['EMA12'] - data['EMA26']
data['Signal'] = data['MACD'].ewm(span=9, adjust=False).mean()
data['Histogram'] = data['MACD'] - data['Signal']

# 매수/매도 포지션
data['MACD Position'] = 0
data.loc[data['MACD'] > data['Signal'], 'MACD Position'] = 1
data.loc[data['MACD'] < data['Signal'], 'MACD Position'] = -1

# 포지션 변화 신호
data['MACD Signal'] = data['MACD Position'].diff()

# 전략 수익률 계산
data['MACD Strategy'] = data['Returns'] * data['MACD Position'].shift(1)
data['Cumulative MACD Returns'] = (1 + data['MACD Strategy']).cumprod()

# 결과 출력
print(data[['일자 / 시간', '종가', 'MACD', 'Signal', 'Histogram', 'MACD Position', 'MACD Signal', 'Returns', 'MACD Strategy', 'Cumulative MACD Returns']].tail())


In [ ]:
import pandas as pd

# 데이터 로드 및 필요한 계산 수행
data = pd.read_csv('/content/엔비디아3분데이터.csv', encoding='cp949')
data['종가'] = data['종가'].astype(float)
data['Returns'] = data['종가'].pct_change()
data['EMA12'] = data['종가'].ewm(span=12, adjust=False).mean()
data['EMA26'] = data['종가'].ewm(span=26, adjust=False).mean()
data['MACD'] = data['EMA12'] - data['EMA26']
data['Signal'] = data['MACD'].ewm(span=9, adjust=False).mean()
data['Histogram'] = data['MACD'] - data['Signal']
data['MACD Position'] = 0
data.loc[data['MACD'] > data['Signal'], 'MACD Position'] = 1
data.loc[data['MACD'] < data['Signal'], 'MACD Position'] = -1
data['MACD Signal'] = data['MACD Position'].diff()
data['MACD Strategy'] = data['Returns'] * data['MACD Position'].shift(1)
data['Cumulative MACD Returns'] = (1 + data['MACD Strategy']).cumprod()

# 결과를 CSV 파일로 저장
output_filename = '/content/MACD_Backtesting_Results.csv'
data[['일자 / 시간', '종가', 'MACD', 'Signal', 'Histogram', 'MACD Position', 'MACD Signal', 'Returns', 'MACD Strategy', 'Cumulative MACD Returns']].to_csv(output_filename, index=False)
